In [1]:
%pip install langchain_groq langchain_core langchain_community pypdf chromadb sentence_transformers gradio tf_keras ipywidgets vaderSentiment gradio

  Obtaining dependency information for langchain_groq from https://files.pythonhosted.org/packages/ca/d8/7166413d50ad06c84a5f59e5fe097cc1c3e05f5aed594d8e42d247b07aa6/langchain_groq-0.3.1-py3-none-any.whl.metadata
  Obtaining dependency information for langchain_core from https://files.pythonhosted.org/packages/57/1e/a65339d128f2e957440db21746f66aaf103d34042fcd130c1801da00492d/langchain_core-0.3.47-py3-none-any.whl.metadata
  Obtaining dependency information for langchain_community from https://files.pythonhosted.org/packages/ab/4b/2652cfd2baa482cb3cdbec1ccccae1674418b7576f21ba7724d8730de9db/langchain_community-0.3.20-py3-none-any.whl.metadata
  Obtaining dependency information for pypdf from https://files.pythonhosted.org/packages/0b/27/d83f8f2a03ca5408dc2cc84b49c0bf3fbf059398a6a2ea7c10acfe28859f/pypdf-5.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/28/8e/5c186c77bf749b6fe0528385e507e463f1667543328d76fd00a49e1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
anaconda-cloud-auth 0.1.4 requires pydantic<2.0, but you have pydantic 2.10.6 which is incompatible.


In [10]:
import os
import json
from collections import deque
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from langchain_groq import ChatGroq
import datetime
import random
import gradio as gr

# -------------------------------
# Helper Functions & Global Setup
# -------------------------------

# File to store user data
USER_DATA_FILE = "user_data.json"

# University mental health resources
UNIVERSITY_RESOURCES = {
    "Centennial College": "Visit the Student Wellness Centre: https://www.centennialcollege.ca/student-health",
    "University of Toronto": "Check U of T’s mental health services: https://mentalhealth.utoronto.ca/",
}

# Motivational quotes for students
MOTIVATIONAL_QUOTES = [
    "Stay focused! Every small step brings you closer to success. 💪",
    "You’re capable of amazing things—keep pushing forward!",
    "Don’t let stress take over! Take breaks, breathe, and keep going. 🚀"
]

def load_user_data():
    if os.path.exists(USER_DATA_FILE):
        with open(USER_DATA_FILE, "r") as file:
            return json.load(file)
    return {}

def save_user_data(user_data):
    with open(USER_DATA_FILE, "w") as file:
        json.dump(user_data, file, indent=4)

def update_user_data(user_id, key, value):
    user_data = load_user_data()
    if user_id not in user_data:
        user_data[user_id] = {}
    user_data[user_id][key] = value
    save_user_data(user_data)

def update_student_profile(user_id, major, year_of_study, common_stressors, university):
    user_data = load_user_data()
    if user_id not in user_data:
        user_data[user_id] = {}
    user_data[user_id]["major"] = major
    user_data[user_id]["year_of_study"] = year_of_study
    user_data[user_id]["common_stressors"] = common_stressors
    user_data[user_id]["university"] = university
    save_user_data(user_data)

def get_mental_health_resources(user_id):
    user_data = load_user_data()
    university = user_data.get(user_id, {}).get("university", "your university")
    if university in UNIVERSITY_RESOURCES:
        return f"If you need support, check out {UNIVERSITY_RESOURCES[university]}"
    else:
        return "I recommend checking your university's website for student wellness resources."

def analyze_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = analyzer.polarity_scores(text)
    compound_score = sentiment_scores["compound"]
    if compound_score <= -0.5:
        return "sad"
    elif -0.5 < compound_score <= -0.1:
        return "frustrated"
    elif -0.1 < compound_score < 0.1:
        return "neutral"
    elif 0.1 <= compound_score < 0.5:
        return "happy"
    else:
        return "excited"

def load_llm():
    return ChatGroq(
        temperature=0.2,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )

def generate_summary(conversation_history, llm):
    prompt = f"""
    You are a helpful assistant. Summarize this conversation in 1-2 sentences.
    
    Conversation:
    {" ".join(conversation_history)}
    
    Summary:
    """
    response = llm.invoke(prompt)
    return response.content.strip()

def check_deadlines(user_id):
    user_data = load_user_data()
    deadlines = user_data.get(user_id, {}).get("deadlines", {})
    today = datetime.date.today()
    upcoming = [
        task for task, date in deadlines.items()
        if datetime.date.fromisoformat(date) <= today + datetime.timedelta(days=3)
    ]
    if upcoming:
        return f"Reminder! You have upcoming deadlines: {', '.join(upcoming)}. Don’t forget to plan ahead!"
    return "No major deadlines soon. Keep up the good work!"

def send_daily_motivation():
    return random.choice(MOTIVATIONAL_QUOTES)

# -----------------------------------------
# Chatbot Function for Gradio Interface
# -----------------------------------------

def chatbot_response(user_message, history, user_id):
    """
    This function takes the latest user message, the conversation history,
    and the user_id. It constructs a conversation prompt (as a string) for the LLM,
    updates the internal history, and returns the UI history in the expected format.
    """
    # Initialize the LLM
    llm = load_llm()
    
    # Load stored user data
    user_data = load_user_data()
    major = user_data.get(user_id, {}).get("major", "student")
    
    # Build the conversation as a list of message dictionaries
    messages = []
    
    # Add a system message for overall context
    messages.append({
        "role": "system",
        "content": (
            f"You are a mental health assistant for students. The user is studying {major}.\n"
            f"- Name: {user_id}\n"
            f"- Last emotion: {user_data.get(user_id, {}).get('last_emotion', 'None')}\n"
            f"- Last conversation: {user_data.get(user_id, {}).get('last_conversation', 'None')}\n\n"
            "Respond with empathy and helpful advice."
        )
    })
    
    # Add the conversation history
    for pair in history:
        user_text, bot_text = pair
        messages.append({"role": "user", "content": user_text})
        messages.append({"role": "assistant", "content": bot_text})
    
    # Add the new user message
    messages.append({"role": "user", "content": user_message})
    
    # Convert the list of message dictionaries to a single prompt string.
    prompt_str = ""
    for msg in messages:
        role = msg["role"].capitalize()
        prompt_str += f"{role}: {msg['content']}\n"
    
    # Invoke the LLM with the concatenated prompt string
    response = llm.invoke(prompt_str)
    bot_reply = response.content.strip()
    
    # Update the internal conversation history (list of tuples)
    history.append((user_message, bot_reply))
    
    # Convert history into the UI format (list of dictionaries with "role" and "content")
    ui_history = []
    for pair in history:
        ui_history.append({"role": "user", "content": pair[0]})
        ui_history.append({"role": "assistant", "content": pair[1]})
    
    return ui_history, history

# ----------------------------
# Gradio Blocks UI Definition
# ----------------------------

with gr.Blocks() as demo:
    gr.Markdown("# Mental Health Chatbot")
    
    with gr.Tab("User Setup"):
        gr.Markdown("### Set up your profile")
        user_name = gr.Textbox(label="Enter your name")
        major = gr.Textbox(label="Your major")
        year_of_study = gr.Textbox(label="Year of Study")
        common_stressors = gr.Textbox(label="Common stressors")
        university = gr.Textbox(label="University")
        setup_button = gr.Button("Set Up Profile")
        setup_output = gr.Textbox(label="Setup Status", interactive=False)
    
    with gr.Tab("Chat"):
        gr.Markdown("### Chat with the Bot")
        chatbot = gr.Chatbot(label="Conversation", type="messages")
        msg = gr.Textbox(label="Your Message")
        clear = gr.Button("Clear Chat")
    
    # Gradio States to hold conversation history and the current user id
    state = gr.State([])
    user_id_state = gr.State("")
    
    def setup_profile(name, major_val, year, stressors, univ):
        """
        Save the profile to our JSON file and update the user_id state.
        """
        if not name.strip():
            return "Please enter a valid name.", ""
        update_user_data(name, "name", name)
        update_student_profile(name, major_val, year, stressors, univ)
        status = f"Profile set up for {name}. Welcome!"
        return status, name

    setup_button.click(
        setup_profile, 
        inputs=[user_name, major, year_of_study, common_stressors, university], 
        outputs=[setup_output, user_id_state]
    )
    
    def respond(message, chat_history, user_id):
        return chatbot_response(message, chat_history, user_id)
    
    msg.submit(respond, [msg, state, user_id_state], [chatbot, state])
    clear.click(lambda: ([], []), None, [chatbot, state])

# Launch the Gradio demo!
demo.launch()


* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.
